In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from helpers import *
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import pickle as pkl
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
from tqdm import tqdm
from datetime import datetime
import seaborn as sns
from sklearn.cluster import KMeans
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
pd.set_option('display.max_columns', None)
import torch
torch.manual_seed(0)
import random
random.seed(0)
import numpy as np
np.random.seed(0)

In [2]:
save_dir = ''
file_names = glob.glob(os.path.join(save_dir, f'QCEH_data/TCV_DATAno*.parquet'))

In [3]:
precomputed_dir = 'precomputed_data/'

List of dataframes (one discharge per each element of the list)

In [4]:
df_list = [pd.read_parquet(x) for x in file_names]
len(df_list)

60

# Initial Analysis - necessary values
For this project we are only focusing on the machine inputs provided in the data.

In [5]:
GENERAL_INPUTS = ["shotnumber", "time"]
MACHINE_INPUTS = ["isbaffled", "ip","b0","nel","ptot","pdiv","q95","betan","kappa","deltaavg","deltaupp",
                  "deltalow","gapin","gapout","zmag","rmag","rmin","lpar_ot","zeff"]
LABEL = ["LHD_label"]
INPUTS = GENERAL_INPUTS + MACHINE_INPUTS + LABEL
len(INPUTS)

22

In [6]:
# This normalizes the X values.
X_train, y_train, column_names = get_training_data(include_time = False, include_shotnumber = False, shot_indices = list(range(55)))
X_test, y_test, _ = get_training_data(include_time = False, include_shotnumber = False, shot_indices = list(range(56, 60)))
X_train.shape

(571675, 19)

In [7]:
D = 40
num_train = X_train.shape[0]
mask = np.arange(num_train) % D == 0
X_train = X_train[mask, :]
y_train = y_train[mask]

In [8]:
X_tensor_train = torch.tensor(X_train, dtype=torch.float32)
y_tensor_train = torch.tensor((y_train - 1).astype(int), dtype=torch.long)

X_tensor_test = torch.tensor(X_test, dtype=torch.float32)
y_tensor_test = torch.tensor((y_test - 1).astype(int), dtype=torch.long)

In [9]:
# Convert NumPy arrays to PyTorch tensors
features_tensor = torch.tensor(X_tensor_train, dtype=torch.float32)
labels_tensor = torch.tensor(y_tensor_train, dtype=torch.long)  # Assuming classification labels are integers

# Create a DataLoader for batching
dataset = TensorDataset(features_tensor, labels_tensor)
batch_size = 32  # You can adjust this according to your dataset size
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)


/var/folders/vq/vpp5gf_96v7f5w4frq2kpfn00000gn/T/ipykernel_6284/3614208288.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  features_tensor = torch.tensor(X_tensor_train, dtype=torch.float32)
/var/folders/vq/vpp5gf_96v7f5w4frq2kpfn00000gn/T/ipykernel_6284/3614208288.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels_tensor = torch.tensor(y_tensor_train, dtype=torch.long)  # Assuming classification labels are integers


In [10]:
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, dropout_prob=0.5, noise_level=0.3):
        super(LSTMClassifier, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.noise_level = noise_level

        # Adding dropout to LSTM
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers, batch_first=True, dropout=dropout_prob if num_layers > 1 else 0.0)

        # Dropout layer before the fully connected layer
        self.dropout = nn.Dropout(dropout_prob)

        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Add Gaussian noise to input
        if self.training and self.noise_level > 0:
            noise = torch.randn_like(x) * self.noise_level
            x = x + noise

        # Initialize hidden state with zeros
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
    
        # Forward pass through LSTM layer
        out, _ = self.lstm(x.unsqueeze(1), (h0, c0))
    
        # Apply dropout
        out = self.dropout(out)

        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out

In [11]:
input_size = 19  # Number of features per time step
hidden_size = 64  # Number of LSTM units
num_layers = 1  # Number of LSTM layers
num_classes = 3  
model = LSTMClassifier(input_size, hidden_size, num_layers, num_classes)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [12]:
num_epochs = 100  

for epoch in range(num_epochs):
    show = True
    for inputs, labels in dataloader:
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backpropagation and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if(show):
            show=False
            print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {loss.item():.4f}')
            #print(f'Epoch [{epoch + 1}/{num_epochs}], ')
        

print('Training complete!')


Epoch [1/100], Training Loss: 1.0485
Epoch [2/100], Training Loss: 0.0243
Epoch [3/100], Training Loss: 0.0082
Epoch [4/100], Training Loss: 0.0037
Epoch [5/100], Training Loss: 0.0033
Epoch [6/100], Training Loss: 0.0020
Epoch [7/100], Training Loss: 0.0009
Epoch [8/100], Training Loss: 0.0034
Epoch [9/100], Training Loss: 0.0019
Epoch [10/100], Training Loss: 0.0008
Epoch [11/100], Training Loss: 0.0008
Epoch [12/100], Training Loss: 0.0004
Epoch [13/100], Training Loss: 0.0003
Epoch [14/100], Training Loss: 0.0009
Epoch [15/100], Training Loss: 0.0001
Epoch [16/100], Training Loss: 0.0002
Epoch [17/100], Training Loss: 0.0002
Epoch [18/100], Training Loss: 0.0009
Epoch [19/100], Training Loss: 0.0003
Epoch [20/100], Training Loss: 0.0005
Epoch [21/100], Training Loss: 0.0007
Epoch [22/100], Training Loss: 0.0002
Epoch [23/100], Training Loss: 0.0007
Epoch [24/100], Training Loss: 0.0004
Epoch [25/100], Training Loss: 0.0002
Epoch [26/100], Training Loss: 0.0001
Epoch [27/100], Train

In [13]:
test_dataset = TensorDataset(X_tensor_test, y_tensor_test)
batch_size = 32  # You can adjust this according to your dataset size
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [14]:

def multi_confusion_matrix(y_predicted, y_expected):
    """
    Function constructing a multiclass confusion matrix
    
    :param y_predicted: pd.Series containing the predicted labels
    :param y_expected: pd.Series containing the expected labels
    :return: pd.Dataframe containing the multiclass confusion matrix 
    with columns as true labels and rows are predicted labels
    """
    predicted_labels = y_predicted.unique()
    expected_labels = y_expected.unique()
    
    mtrx = pd.DataFrame(0, columns=expected_labels, index=expected_labels)
    
    for expected_label in expected_labels:
        for predicted_label in predicted_labels:
            count = sum((y_expected == expected_label) & (y_predicted == predicted_label))
            mtrx.at[predicted_label, expected_label] = count
    return mtrx

def diag(matrix):
    """
    Function retrieving the true values of a confusion matrix
    
    :param matrix: confusion matrix
    :return: list of all the diagonal values
    """
    if len(matrix.index) <= len(matrix.columns):
        zipped = zip(matrix.index, matrix.columns[:len(matrix.index)])
    else:
        zipped = zip(matrix.index[:len(matrix.columns)], matrix.columns)
    
    diag = []
    for idx,col in zipped:
        diag.append(matrix.at[idx,col])
    return diag

def multi_accuracy(matrix):
    """
    Function computing the accuracy associated with a confusion matrix
    :param matrix: confusion matrix
    :return: precision score
    """
    t = sum(diag(matrix))
    all = matrix.sum().sum()
    return t / all
    
def multi_precision(matrix):
    """
    Function computing the precision associated with a confusion matrix
    :param matrix: confusion matrix
    :return: precision score
    """
    tp = diag(matrix)
    pred_p = matrix.sum(axis=1)
    return (tp / pred_p).fillna(0)
def multi_recall(matrix):
    """
    Function computing the recall associated with a confusion matrix
    :param matrix: confusion matrix
    :return: recall score
    """
    tp = diag(matrix)
    acutal_p = matrix.sum(axis=0)
    return (tp / acutal_p).fillna(0)

def multi_f1_score(matrix):
    """
    Function computing the F1-score associated with a confusion matrix
    :param matrix: confusion matrix
    :return: F1-score
    """
    recall = multi_recall(matrix)
    precision = multi_precision(matrix)
    
    multiplied = recall.multiply(precision)
    summed =recall + precision
    return (2 * multiplied / summed).fillna(0)

In [15]:
import pandas as pd

model.eval()
all_predicted = []
all_labels = []

with torch.no_grad():
    for shots, labels in test_loader:
        outputs = model(shots)
        _, predicted = torch.max(outputs.data, 1)
        all_predicted.extend(predicted.numpy())
        all_labels.extend(labels.numpy())

# Convert to pandas Series for compatibility with your functions
y_predicted = pd.Series(all_predicted)
y_true = pd.Series(all_labels)

# Assuming your multi_confusion_matrix, multi_f1_score, and multi_accuracy functions accept pandas Series
mtrx_test = multi_confusion_matrix(y_predicted + 1, y_true + 1)  # Adjust indices if necessary
f1_score = multi_f1_score(mtrx_test)
accuracy = multi_accuracy(mtrx_test)

# Print the evaluation metrics
print(f'F1 Score: {f1_score}')
print(f'Accuracy: {accuracy}')


F1 Score: 1    0.911356
3    0.896329
2    0.806564
dtype: float64
Accuracy: 0.8822099396830493


In [16]:
model.eval()

LSTMClassifier(
  (lstm): LSTM(19, 64, batch_first=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=64, out_features=3, bias=True)
)

In [17]:
with open(f"{precomputed_dir}model-19-64-shuffle-falseDropout4-Noise.pkl", 'wb') as f:
    pkl.dump(model, f)